# PPO agent with dietkit data

In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import pandas as pd
from meal_planning_environment import MealPlanningEnv
from dietkit import load_ingredient, load_menu, load_diet



In [2]:
all_ingredients = load_ingredient(sample_language = 'eng')
meal_dict = load_menu(ingredients = all_ingredients, sample_language = 'eng')
example_mealplans = load_diet(menus = meal_dict, num_loads = 100, sample_language = 'eng', sample_name = 'ML')

possible_meals = np.array(list(meal_dict.keys()))
display(possible_meals)

nutrition_data = pd.DataFrame([meal.nutrition for meal in meal_dict.values()], index=possible_meals)
display(nutrition_data)

array(['S watermelon punch (without milks)', 'Dried persimmon',
       'S kyogo grape (100g)', ..., 'Green onion kimchi',
       'Pickled paprika', 'empty'], dtype='<U72')

,Energy,Protein,Fat,Carbohydrate,Total Dietary,Calcium,Iron,Sodium,Vitamin A,Vitamin B1 (Thiamine),Vitamin B2 (Rivoflavin),Vitamin C,Linoleic Acid,Alpha-Linolenic Acid
S watermelon punch (without milks),55.00,0.7900,0.05000,13.8180,0.200,7.200,0.18600,0.600,71.083333,0.024000,0.03000,0.0000,14.2500,2.9300
Dried persimmon,76.50,0.5820,0.04800,20.5830,3.360,6.300,0.19200,0.900,13.725000,0.003000,0.06720,1.5330,0.0000,8.6010
S kyogo grape (100g),61.00,0.4300,0.04000,16.3800,0.400,4.000,0.17000,1.000,2.166667,0.026000,0.04700,0.1300,11.8900,6.0900
S dried persimmon,85.60,0.7720,0.03200,22.9800,3.400,5.600,0.24800,0.400,9.533333,0.003600,0.09720,1.0680,0.0000,5.0440
S fruit punch,101.05,3.6700,3.36450,14.8900,0.790,117.650,0.19850,36.000,91.166667,0.040350,0.19110,1.8925,86.6605,10.6925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ponytail radish kimchi,10.25,0.5050,0.05000,1.9725,0.775,10.500,0.16750,173.000,5.541667,0.003250,0.01900,0.4125,19.7050,9.4775
Kohlrabi watery kimchi,5.50,0.2900,0.01250,1.2900,0.600,10.500,0.06750,1.750,0.250000,0.203750,0.00875,0.0000,1.9650,5.1100
Green onion kimchi,12.25,0.6250,0.14500,2.1275,1.150,15.000,0.18250,163.750,26.854167,0.012750,0.02850,0.5550,79.8050,17.1425
Pickled paprika,7.94,0.2017,0.03005,1.9859,0.348,1.675,0.08145,67.204,4.850000,0.003675,0.02709,24.2820,12.4483,6.5448


In [3]:
num_meals = 21
env = MealPlanningEnv(possible_meals=possible_meals, nutrition_data=nutrition_data, num_meals=num_meals)
model = PPO("MultiInputPolicy", env, verbose=1)
obs = env.reset()
for i in range(num_meals):
  action, _ = model.predict(obs)
  obs, rewards, done, info = env.step(action)
env.render()
model.learn(total_timesteps=1000)

obs = env.reset()
for i in range(num_meals):
  action, _ = model.predict(obs)
  obs, rewards, done, info = env.step(action)
  action, _states = model.predict(obs)
  # print(action)
  
 
env.render()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Step: 21
Chosen Meal: Braised quail eggs in ketchup
Meal History: ['Stir-fried chicken breast in oyster sauce'
 'Braised potato and fried fish paste'
 'Stir-fried sausage and baby anchovy' 'Young pumpkin and onion soup'
 'S yangsongyi mushroom veggie porridge' 'S fried pancake'
 'S cabbage potato porridge' 'S milk (100ml)'
 'Stir-fried seaweed stem and vegetables' 'Cuttlefish daikon soup'
 'Steamed carrot stick'
 'Grilled salted seaweed + wild chive marinade sauce'
 'Teriyaki sauce-braised pork' 'Mandu (dumpling) soup' 'S banana (50g)'
 'Kimchi onion soup' 'Fusilli pasta' 'Pan-fried pollack(sliced)'
 'Beef lettuce bibimbap' 'Seasoned spinach'
 'Braised quail eggs in ketchup']
Nutrition History: [[6.62000e+01 7.25000e+00 2.51000e+00 3.31000e+00 4.40000e-01 6.64000e+00
  2.10000e-01 1.54120e+02 2.72000e+01 7.00000e-02 4.00000e-02 1.47000e+00
  1.15513e+03 1.37020e+02]
 [6.76400e+01 2.41000e+00 2